In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
import os
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
base_dir = '/content/gdrive/My Drive/data'
direction_path = [ os.path.join(base_dir,p)  for p in os.listdir(base_dir)]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
direction_path

['/content/gdrive/My Drive/data/Left_Data',
 '/content/gdrive/My Drive/data/Right_Data',
 '/content/gdrive/My Drive/data/Straight_Data']

In [ ]:
X = []
Y = []
for i,path in enumerate(direction_path):
  j = 0
  for img in os.listdir(path):
    
    if j == 1000:
      break
      
    i = i%3
    
    img_path = os.path.join(path,img)
    IMG = cv2.imread(img_path)
    IMG = cv2.resize(IMG, (128,128))
    IMG = np.array(IMG).astype('float32') / 255.
    X.append(IMG)
    Y.append(i)
    j+=1

In [ ]:
X1 = np.array(X)
Y1 = np.array(Y)

from keras.utils import to_categorical

Y1 = to_categorical(Y1 , num_classes = 3)

print(X1.shape)
print(Y1.shape)

(3000, 128, 128, 3)
(3000, 3)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test , y_train, y_test = train_test_split(X1, Y1, test_size = 0.2)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2400, 128, 128, 3)
(2400, 3)
(600, 128, 128, 3)
(600, 3)


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Flatten
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Conv2D(16 , kernel_size = (3,3) , strides = (2,2) , padding = 'same' , activation = 'relu', input_shape = (128,128,3)))
model.add(MaxPooling2D(pool_size = (2,2)))  # 64x64x16
model.add(BatchNormalization())

model.add(Conv2D(32 , kernel_size = (3,3) , strides = (2,2) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))  # 32X32x32
model.add(BatchNormalization())

model.add(Conv2D(32 , kernel_size = (3,3) , strides = (2,2) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))  # 16x16x32
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024 , activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(512 , activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(256 , activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(128 , activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(128 , activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(3 , activation = 'softmax'))





In [ ]:
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 5, validation_split = 0.1, batch_size = 16)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2160 samples, validate on 240 samples
Epoch 1/5
2160/2160 [==============================] - 10s 5ms/step - loss: 0.5446 - acc: 0.7875 - val_loss: 0.7360 - val_acc: 0.8083
Epoch 2/5
2160/2160 [==============================] - 8s 4ms/step - loss: 0.3744 - acc: 0.8731 - val_loss: 0.5208 - val_acc: 0.8375
Epoch 3/5
2160/2160 [==============================] - 8s 4ms/step - loss: 0.3139 - acc: 0.9000 - val_loss: 0.2965 - val_acc: 0.9000
Epoch 4/5
2160/2160 [==============================] - 9s 4ms/step - loss: 0.2653 - acc: 0.9167 - val_loss: 0.3919 - val_acc: 0.8792
Epoch 5/5
2160/2160 [==============================] - 9s 4ms/step - loss: 0.2211 - acc: 0.9213 - val_loss: 0.2887 - val_acc: 0.8875


In [ ]:
model.evaluate(X_test,y_test)

600/600 [==============================] - 1s 1ms/step


[0.27738917072614033, 0.8999999992052714]

In [ ]:
model.predict_classes(X_test)

array([2, 2, 0, 2, 1, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 1, 1,
       1, 0, 2, 2, 2, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 2, 1, 2, 2, 0, 0,
       0, 1, 0, 1, 0, 2, 2, 1, 2, 1, 1, 0, 1, 2, 1, 2, 0, 2, 0, 0, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 2,
       2, 0, 0, 2, 0, 0, 0, 2, 2, 1, 2, 0, 0, 0, 2, 1, 2, 2, 1, 0, 1, 2,
       2, 0, 2, 2, 1, 1, 1, 1, 0, 0, 1, 1, 2, 0, 2, 0, 0, 2, 1, 0, 0, 2,
       2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 1, 0, 2, 1, 2, 0, 1, 2, 2, 1,
       1, 2, 2, 1, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 1, 1, 0, 0, 2, 1, 2,
       0, 0, 2, 0, 1, 2, 0, 2, 1, 2, 0, 0, 0, 1, 1, 1, 2, 0, 2, 0, 2, 1,
       0, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 1, 1, 2, 1, 0,
       2, 1, 1, 2, 1, 0, 0, 0, 0, 2, 2, 2, 1, 0, 1, 2, 0, 1, 1, 2, 2, 1,
       1, 2, 2, 1, 2, 0, 0, 1, 2, 2, 0, 1, 0, 0, 2, 0, 0, 2, 1, 0, 1, 2,
       2, 2, 0, 1, 1, 2, 1, 2, 2, 0, 2, 0, 1, 1, 2, 2, 1, 1, 2, 0, 0, 2,
       0, 1, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 1, 1,

In [ ]:
model.save("model.h7")
